In [1]:
from pyflink.table import EnvironmentSettings, TableEnvironment

In [2]:
env_settings =  EnvironmentSettings.in_streaming_mode()
table_env =  TableEnvironment.create(env_settings)

Could not find a compatible bson package.


In [4]:
from pyflink.table import EnvironmentSettings,TableEnvironment
env_settings = EnvironmentSettings.in_batch_mode()
table_env = TableEnvironment.create(env_settings)

table = table_env.from_elements([(1,"hyd"),(2,"chn"),(3,"mum")])
table.execute().print()

+----------------------+--------------------------------+
|                   _1 |                             _2 |
+----------------------+--------------------------------+
|                    1 |                            hyd |
|                    2 |                            chn |
|                    3 |                            mum |
+----------------------+--------------------------------+
3 rows in set


In [5]:
from pyflink.table import DataTypes

In [8]:
table = table_env.from_elements([(1,"hyd"),(2,"chn"),(3,"mum")],
                               DataTypes.ROW([DataTypes.FIELD("id",DataTypes.TINYINT()),
                                             DataTypes.FIELD("city",DataTypes.STRING())]) )
table.execute().print()

+--------+--------------------------------+
|     id |                           city |
+--------+--------------------------------+
|      1 |                            hyd |
|      2 |                            chn |
|      3 |                            mum |
+--------+--------------------------------+
3 rows in set


In [9]:
from pyflink.table import EnvironmentSettings, TableEnvironment

# create a stream TableEnvironment
env_settings = EnvironmentSettings.in_streaming_mode()
table_env = TableEnvironment.create(env_settings)

In [10]:
table_env.execute_sql("""
    CREATE TABLE random_source (
        id BIGINT, 
        data TINYINT 
    ) WITH (
        'connector' = 'datagen',
        'fields.id.kind'='sequence',
        'fields.id.start'='1',
        'fields.id.end'='3',
        'fields.data.kind'='sequence',
        'fields.data.start'='4',
        'fields.data.end'='6'
    )
""")
table = table_env.from_path("random_source")
table.execute().print()

+----+----------------------+--------+
| op |                   id |   data |
+----+----------------------+--------+
| +I |                    1 |      4 |
| +I |                    2 |      5 |
| +I |                    3 |      6 |
+----+----------------------+--------+
3 rows in set


In [12]:
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.expressions import call, col

# using batch table environment to execute the queries
env_settings = EnvironmentSettings.in_batch_mode()
table_env = TableEnvironment.create(env_settings)

orders = table_env.from_elements([('Jack', 'FRANCE', 10), ('Rose', 'ENGLAND', 30), ('Jack', 'FRANCE', 20)],
                                 ['name', 'country', 'revenue'])


revenue = orders.select(col('name'),col('country'),col('revenue')).execute().print()

+--------------------------------+--------------------------------+----------------------+
|                           name |                        country |              revenue |
+--------------------------------+--------------------------------+----------------------+
|                           Jack |                         FRANCE |                   10 |
|                           Rose |                        ENGLAND |                   30 |
|                           Jack |                         FRANCE |                   20 |
+--------------------------------+--------------------------------+----------------------+
3 rows in set


In [13]:
revenue = orders.select(col('name'),col('country'),col('revenue')).where(col('country')=='FRANCE')\
.group_by(col('name')).select(col('name'),call("sum",col('revenue')).alias('revenue'))

revenue.execute().print()

+--------------------------------+----------------------+
|                           name |              revenue |
+--------------------------------+----------------------+
|                           Jack |                   30 |
+--------------------------------+----------------------+
1 row in set


In [15]:
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table import DataTypes
from pyflink.table.udf import udf
import pandas as pd

# using batch table environment to execute the queries
env_settings = EnvironmentSettings.in_batch_mode()
table_env = TableEnvironment.create(env_settings)
orders = table_env.from_elements([('Jack', 'FRANCE', 10), ('Rose', 'ENGLAND', 30), ('Jack', 'FRANCE', 20)],
                                 ['name', 'country', 'revenue'])

In [16]:
map_function = udf(lambda x: pd.concat([x.name, x.revenue * 10], axis=1),
                   result_type=DataTypes.ROW(
                               [DataTypes.FIELD("name", DataTypes.STRING()),
                                DataTypes.FIELD("revenue", DataTypes.BIGINT())]),
                   func_type="pandas")

In [17]:
orders.map(map_function).execute().print()

+--------------------------------+----------------------+
|                           name |              revenue |
+--------------------------------+----------------------+
|                           Jack |                  100 |
|                           Rose |                  300 |
|                           Jack |                  200 |
+--------------------------------+----------------------+
3 rows in set


In [18]:
from pyflink.table import EnvironmentSettings, TableEnvironment

# use a stream TableEnvironment to execute the queries
env_settings = EnvironmentSettings.in_streaming_mode()
table_env = TableEnvironment.create(env_settings)

In [33]:
table_env.execute_sql("""
    CREATE TABLE random_source3 (
        id BIGINT, 
        data TINYINT
    ) WITH (
        'connector' = 'datagen',
        'fields.id.kind'='sequence',
        'fields.id.start'='1',
        'fields.id.end'='8',
        'fields.data.kind'='sequence',
        'fields.data.start'='4',
        'fields.data.end'='11'
    )
""")

table_env.execute_sql("""
    CREATE TABLE print_sink2 (
        id BIGINT, 
        data_sum TINYINT 
    ) WITH (
        'connector' = 'datagen'
        )
""")

In [37]:
table_env.execute_sql("""
insert into print_sink
select id, sum(data) from random_source group by id
""").wait()

4> +I[8, 11]
8> +I[3, 6]
1> +I[7, 10]
2> +I[4, 7]
6> +I[1, 4]
7> +I[6, 9]
6> +I[2, 5]
2> +I[5, 8]


In [38]:
table_env.sql_query('select * from random_source2').execute().print()

+----+----------------------+--------+
| op |                   id |   data |
+----+----------------------+--------+
| +I |                    1 |      4 |
| +I |                    2 |      5 |
| +I |                    3 |      6 |
| +I |                    4 |      7 |
| +I |                    5 |      8 |
| +I |                    6 |      9 |
| +I |                    7 |     10 |
| +I |                    8 |     11 |
+----+----------------------+--------+
8 rows in set


In [40]:
table_env.sql_query('select * from random_source2').to_pandas()

,id,data
0,1,4
1,2,5
2,3,6
3,4,7
4,5,8
5,6,9
6,7,10
7,8,11


In [1]:
from pyflink.table import *
from pyflink.table.expressions import col

# environment configuration
t_env = TableEnvironment.create(
    environment_settings=EnvironmentSettings.in_batch_mode())

source_data_path = "source/"
result_data_path = "sink/"

Could not find a compatible bson package.


In [2]:
source_ddl = f"""
        create table Orders(
            a VARCHAR,
            b BIGINT,
            c BIGINT
        ) with (
            'connector' = 'filesystem',
            'format' = 'csv',
            'path' = '{source_data_path}'
        )
        """
t_env.execute_sql(source_ddl)

In [3]:
sink_ddl = f"""
    create table `Result`(
        a VARCHAR,
        cnt BIGINT
    ) with (
        'connector' = 'filesystem',
        'format' = 'csv',
        'path' = '{result_data_path}'
    )
    """
t_env.execute_sql(sink_ddl)

In [4]:
orders = t_env.from_path("Orders")

In [29]:
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table import DataTypes
from pyflink.table.udf import udf
import pandas as pd

# using batch table environment to execute the queries
env_settings = EnvironmentSettings.in_batch_mode()
table_env = TableEnvironment.create(env_settings)
orders = table_env.from_elements([('Jack', 'FRANCE', 10), ('Rose', 'ENGLAND', 30), ('Jack', 'FRANCE', 20), ('Jack', 'FRANCE', 30)],
                                 ['name', 'country', 'revenue'])

In [30]:
orders.execute().print()

+--------------------------------+--------------------------------+----------------------+
|                           name |                        country |              revenue |
+--------------------------------+--------------------------------+----------------------+
|                           Jack |                         FRANCE |                   10 |
|                           Rose |                        ENGLAND |                   30 |
|                           Jack |                         FRANCE |                   20 |
|                           Jack |                         FRANCE |                   30 |
+--------------------------------+--------------------------------+----------------------+
4 rows in set


In [31]:
orders.group_by(col('country')).select(col('country'),col('revenue').count.alias('cnt')).execute().print()

+--------------------------------+----------------------+
|                        country |                  cnt |
+--------------------------------+----------------------+
|                        ENGLAND |                    1 |
|                         FRANCE |                    3 |
+--------------------------------+----------------------+
2 rows in set


In [32]:
result =  orders.filter(col('country').is_not_null & col('revenue').is_not_null ).select(col('country').lower_case.alias('country')\
                                                                                        ,col('revenue'),col('name')).execute().print()

+--------------------------------+----------------------+--------------------------------+
|                        country |              revenue |                           name |
+--------------------------------+----------------------+--------------------------------+
|                         france |                   10 |                           Jack |
|                        england |                   30 |                           Rose |
|                         france |                   20 |                           Jack |
|                         france |                   30 |                           Jack |
+--------------------------------+----------------------+--------------------------------+
4 rows in set


In [33]:
orders.execute().print()

+--------------------------------+--------------------------------+----------------------+
|                           name |                        country |              revenue |
+--------------------------------+--------------------------------+----------------------+
|                           Jack |                         FRANCE |                   10 |
|                           Rose |                        ENGLAND |                   30 |
|                           Jack |                         FRANCE |                   20 |
|                           Jack |                         FRANCE |                   30 |
+--------------------------------+--------------------------------+----------------------+
4 rows in set


In [34]:
from pyflink.table.expressions import concat
orders = orders.add_columns(concat(col("country"), 'sunny').alias('4thcol'))
orders.execute().print()

+--------------------------------+--------------------------------+----------------------+--------------------------------+
|                           name |                        country |              revenue |                         4thcol |
+--------------------------------+--------------------------------+----------------------+--------------------------------+
|                           Jack |                         FRANCE |                   10 |                    FRANCEsunny |
|                           Rose |                        ENGLAND |                   30 |                   ENGLANDsunny |
|                           Jack |                         FRANCE |                   20 |                    FRANCEsunny |
|                           Jack |                         FRANCE |                   30 |                    FRANCEsunny |
+--------------------------------+--------------------------------+----------------------+--------------------------------+
4 rows i

In [36]:
orders.add_or_replace_columns(concat(col("4thcol"), 'rainy').alias('4thcol')).execute().print()

+--------------------------------+--------------------------------+----------------------+--------------------------------+
|                           name |                        country |              revenue |                         4thcol |
+--------------------------------+--------------------------------+----------------------+--------------------------------+
|                           Jack |                         FRANCE |                   10 |               FRANCEsunnyrainy |
|                           Rose |                        ENGLAND |                   30 |              ENGLANDsunnyrainy |
|                           Jack |                         FRANCE |                   20 |               FRANCEsunnyrainy |
|                           Jack |                         FRANCE |                   30 |               FRANCEsunnyrainy |
+--------------------------------+--------------------------------+----------------------+--------------------------------+
4 rows i

In [38]:
orders =  orders.drop_columns(col('4thcol'))
orders.execute().print()

+--------------------------------+--------------------------------+----------------------+
|                           name |                        country |              revenue |
+--------------------------------+--------------------------------+----------------------+
|                           Jack |                         FRANCE |                   10 |
|                           Rose |                        ENGLAND |                   30 |
|                           Jack |                         FRANCE |                   20 |
|                           Jack |                         FRANCE |                   30 |
+--------------------------------+--------------------------------+----------------------+
4 rows in set


In [42]:
orders1 = orders.select(col('name').alias('name1'),col('country').alias('country1'),col('revenue').alias('revenue1'))
orders2 = orders

In [43]:
orders1.join(orders2).where(col('name1')==col('name')).select(col('name'),col('name1'),col('country'),col('country1')).execute().print()

+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                           name |                          name1 |                        country |                       country1 |
+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                           Jack |                           Jack |                         FRANCE |                         FRANCE |
|                           Jack |                           Jack |                         FRANCE |                         FRANCE |
|                           Jack |                           Jack |                         FRANCE |                         FRANCE |
|                           Jack |                           Jack |                         FRANCE |                         FRANCE |
|                           Jack |                           J

In [44]:
orders1.left_outer_join(orders2).where(col('name1')==col('name')).select(col('name'),col('name1'),col('country'),col('country1')).execute().print()

+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                           name |                          name1 |                        country |                       country1 |
+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                           Jack |                           Jack |                         FRANCE |                         FRANCE |
|                           Jack |                           Jack |                         FRANCE |                         FRANCE |
|                           Jack |                           Jack |                         FRANCE |                         FRANCE |
|                           Jack |                           Jack |                         FRANCE |                         FRANCE |
|                           Jack |                           J

In [46]:
orders1.right_outer_join(orders2).where(col('name1')==col('name')).select(col('name'),col('name1'),col('country'),col('country1')).execute().print()

TypeError: Table.right_outer_join() missing 1 required positional argument: 'join_predicate'